# Подготовим данные для тренировки модели

Нам даны две таблицы: в первой собраны данные о действиях пользователей, во второй - данные о попытках пользователей решить задания. Импортируем библиотеку Pandas и посмотрим на формат наших данных:

In [143]:
import pandas as pd
events_data = pd.read_csv('datasets/event_data_train.csv')
submissions_data = pd.read_csv('datasets/submissions_data_train.csv')

In [144]:
events_data.head()

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


1. step_id - id стэпа
2. user_id - анонимизированный id юзера
3. timestamp - время наступления события в формате unix date
4. action - событие, возможные значения: 

* discovered - пользователь перешел на стэп
* viewed - просмотр шага,
* started_attempt - начало попытки решить шаг
* passed - удачное решение практического шага

In [145]:
submissions_data.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


1. step_id - id стэпа
2. timestamp - время отправки решения в формате unix date
3. submission_status - статус решения
4. user_id - анонимизированный id юзера

Так как мы рассматриваем прогресс каждого студента в общем, а не по каждому степу, уберем колонку step_id из таблиц:

In [146]:
events_data = events_data.drop('step_id', axis=1)
submissions_data = submissions_data.drop('step_id', axis=1)

Теперь посчитаем количество различных событий и количество верных и неверных решений для каждого студента. Применим метод pivot_table к нашим таблицам:

In [147]:
action_count = events_data.pivot_table(values='action', index='user_id', 
                                       columns='action', aggfunc='count').fillna(0)

submissions_count = submissions_data.pivot_table(values='submission_status', 
                                                 index='user_id',
                                                 columns='submission_status', 
                                                 aggfunc='count').fillna(0)

Также посчитаем такой показатель, как отношение правильных ответов ко всем совершенным попыткам:

In [148]:
submissions_count['correct_ratio'] = submissions_count.correct / \
(submissions_count.correct + submissions_count.wrong)

In [149]:
action_count.head()

action,discovered,passed,started_attempt,viewed
user_id,,,,
1,1.0,0.0,0.0,1.0
2,9.0,9.0,2.0,10.0
3,91.0,87.0,30.0,192.0
5,11.0,11.0,4.0,12.0
7,1.0,1.0,0.0,1.0


In [150]:
submissions_count.head()

submission_status,correct,wrong,correct_ratio
user_id,,,
2,2.0,0.0,1.000000
3,29.0,23.0,0.557692
5,2.0,2.0,0.500000
8,9.0,21.0,0.300000
14,0.0,1.0,0.000000


Чтобы собрать все необходимые данные в одном месте, соединим две таблицы в одну:

In [151]:
complete_data = action_count.join(submissions_count, 
                                  on='user_id', how='outer').fillna(0)

Добавим в новую таблицу колонку, которая отображает, прошел пользователь курс или нет. По условию задачи мы считаем, что пользователь прошел курс, если он прошел как минимум 40 заданий:

In [152]:
complete_data['passed_course'] = (complete_data['correct'] > 40).astype('int')

In [171]:
complete_data.head()

,discovered,passed,started_attempt,viewed,correct,wrong,correct_ratio,passed_course
user_id,,,,,,,,
1,1.0,0.0,0.0,1.0,0.0,0.0,0.000000,0
2,9.0,9.0,2.0,10.0,2.0,0.0,1.000000,0
3,91.0,87.0,30.0,192.0,29.0,23.0,0.557692,0
5,11.0,11.0,4.0,12.0,2.0,2.0,0.500000,0
7,1.0,1.0,0.0,1.0,0.0,0.0,0.000000,0


Наша таблица для тренировки модели почти готова! Осталось только для каждого пользователя выбрать те записи, которые отображают прогресс пользователя за первые 2 дня прохождения курса.

Для этого в исходных таблицах найдем первый, минимальный таймстэмп пользователя, затем прибавим к нему 2 дня (48\*60\*60 секунд), и возьмем те записи, которые будут принадлежать этому временному промежутку:

In [154]:
events_min_max_time = pd.DataFrame(events_data.groupby('user_id').
                                   timestamp.min()). \
                                   rename(columns={'timestamp': 'min_time'})

events_min_max_time['max_time'] = events_min_max_time.min_time + 48*60*60
events_data = events_data.join(events_min_max_time, on='user_id')

submissions_min_max_time = pd.DataFrame(submissions_data.groupby('user_id').
                                        timestamp.min()). \
                                        rename(columns={'timestamp': 'min_time'})
    
submissions_min_max_time['max_time'] = submissions_min_max_time.min_time + 48*60*60
submissions_data = submissions_data.join(submissions_min_max_time, on='user_id')

events_2days = events_data.loc[events_data.timestamp <= events_data.max_time]
submissions_2days = submissions_data.loc[submissions_data.timestamp <= 
                                         submissions_data.max_time]

Мы извлекли нужные записи - теперь нужно привести их к такому же виду, к какому мы привели исходные таблицы:

In [155]:
events_2days = events_2days.drop(['min_time', 'max_time'], axis=1)
submissions_2days = submissions_2days.drop(['min_time', 'max_time'], axis=1)

action_count_2days = events_2days.pivot_table(values='action', 
                                              index='user_id', 
                                              columns='action', 
                                              aggfunc='count').fillna(0)

submissions_count_2days = submissions_2days.pivot_table(values='submission_status', 
                                                        index='user_id',
                                                        columns='submission_status', 
                                                        aggfunc='count').fillna(0)

submissions_count_2days['correct_ratio'] = submissions_count_2days.correct / \
                                           (submissions_count_2days.correct + 
                                            submissions_count_2days.wrong)

data_2days = action_count_2days.join(submissions_count_2days, 
                                     on='user_id', how='outer').fillna(0)

И финальный шаг: добавим к двухдневной таблице колонку "из будущего" - прошел ли пользователь в итоге курс или нет. Это колонка *passed_course*, которую мы вычислили на основе полных данных.

In [156]:
data_2days = data_2days.join(complete_data.passed_course, on='user_id')

In [157]:
data_2days.head()

,discovered,passed,started_attempt,viewed,correct,wrong,correct_ratio,passed_course
user_id,,,,,,,,
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0
2,9.0,9.0,2.0,9.0,2.0,0.0,1.0,0
3,15.0,15.0,4.0,20.0,4.0,4.0,0.5,0
5,1.0,1.0,0.0,1.0,2.0,2.0,0.5,0
7,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0


Прекрасно! Теперь у нас есть таблица, в которой отслеживается прогресс пользователя за первые два дня прохождения курса, но также в этой таблице есть информация о том, прошел ли пользователь в итоге курс или нет.

На такой таблице мы и будем обучать нашу модель. Сохраним наши обработанные данные в CSV формате и перейдем к следующему шагу.

In [158]:
data_2days.to_csv('datasets/ML_Contest_train.csv')

# Подготовка тестового датасета 

Загрузим наши тестовые данные - те данные, на основе которых и нужно предсказать, получит ли пользователь заветный сертификат.

In [159]:
events_data_test = pd.read_csv('datasets/events_data_test.csv')
submissions_data_test = pd.read_csv('datasets/submission_data_test.csv')

In [160]:
events_data_test.head()

,step_id,timestamp,action,user_id
0,30456,1526893787,viewed,24417
1,30456,1526893797,viewed,24417
2,30456,1526893954,viewed,24417
3,30456,1526895780,viewed,24417
4,30456,1526893787,discovered,24417


In [161]:
submissions_data_test.head()

,step_id,timestamp,submission_status,user_id
0,31971,1526800961,wrong,24370
1,31971,1526800976,wrong,24370
2,31971,1526800993,wrong,24370
3,31971,1526801054,correct,24370
4,31972,1526800664,wrong,24370


Эти данные записаны в таком же сыром виде, как и сырые данные для нашей тренировочной модели. Поэтому проведем для них те же операции, чтобы привести к нужному формату:

In [162]:
events_data_test = events_data_test.drop('step_id', axis=1)
submissions_data_test = submissions_data_test.drop('step_id', axis=1)

action_count_test = events_data_test.pivot_table(values='action', 
                                                 index='user_id',
                                                 columns='action', 
                                                 aggfunc='count').fillna(0)

submissions_count_test = submissions_data_test.pivot_table(values='submission_status', 
                                                           index='user_id',
                                                           columns='submission_status', 
                                                           aggfunc='count').fillna(0)

submissions_count_test['correct_ratio'] = submissions_count_test.correct / \
                                         (submissions_count_test.correct + 
                                          submissions_count_test.wrong)

complete_data_test = action_count_test.join(submissions_count_test, 
                                            on='user_id', how='outer').fillna(0)

In [170]:
complete_data_test.head()

,discovered,passed,started_attempt,viewed,correct,wrong,correct_ratio
user_id,,,,,,,
4,1.0,1.0,0.0,1.0,0.0,0.0,0.000000
6,1.0,1.0,0.0,1.0,0.0,0.0,0.000000
10,2.0,2.0,0.0,6.0,0.0,0.0,0.000000
12,11.0,9.0,4.0,14.0,1.0,0.0,1.000000
13,70.0,70.0,35.0,105.0,29.0,36.0,0.446154


# Обучение модели

Настало время машинного обучения! Импортируем Random Forest и RandomizedSearchCV из библиотеки Scikit-learn:

In [164]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

Загрузим наш тренировочный датасет: обозначим зависимую переменную *passed_course* как y, а все независимые переменные как X.

In [165]:
train_dataset = pd.read_csv('datasets/ML_Contest_train.csv', index_col='user_id')
X = train_dataset.drop('passed_course', axis=1)
y = train_dataset.passed_course

Потренируем разные Random Forest и выберем из них лучший с помощью RandomizedSearchCV:

In [172]:
clf = RandomForestClassifier()
params = {
    'n_estimators': range(50, 200),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 10),
    'min_samples_split': range(2, 100),
    'min_samples_leaf': range(1, 50)
}

search = RandomizedSearchCV(clf, params, n_iter=100, n_jobs=-1)
search.fit(X, y)
best_tree = search.best_estimator_
best_tree

RandomForestClassifier(criterion='entropy', max_depth=7, min_samples_leaf=4,
                       min_samples_split=46, n_estimators=111)

Все готово! Давайте предскажем вероятность того, что пользователь успешно закончит курс!

In [173]:
predict = best_tree.predict_proba(complete_data_test)[:, 1]

Результатом работы нашей программы должен стать файл, в котором указан id юзера и соответствующая ему вероятность завершения курса:

In [174]:
result = pd.Series(predict, index=complete_data_test.reset_index().user_id, 
                   name='is_gone')

result.to_csv('predictions.csv')

In [175]:
result.sort_values(ascending=False)

user_id
21685    0.890880
21444    0.878990
7374     0.876534
17982    0.869989
19796    0.856592
           ...   
9471     0.000000
9490     0.000000
9515     0.000000
9518     0.000000
26800    0.000000
Name: is_gone, Length: 6184, dtype: float64

Осталось лишь проверить точность наших предсказаний. Загрузим наш результат в проверочную форму и получим ROC_AUC_SCORE = 0.8888238705469613, что говорит о довольно высокой точности предсказания нашей модели!

![Скриншот результата](result.png)